In [ ]:
import cv2
import numpy as np

In [ ]:
# Load YOLOv3 weights and configuration
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

In [ ]:
# Load COCO class names
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

In [ ]:
# Get output layer names
layer_names = net.getUnconnectedOutLayersNames()

In [ ]:
# Open video stream (0 for default camera, or provide a video file path)
cap = cv2.VideoCapture(1)

In [ ]:
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame for YOLOv3
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Forward pass through the network
    detections = net.forward(layer_names)

    # Post-process the detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Adjust this threshold as needed
                center_x = int(obj[0] * frame.shape[1])
                center_y = int(obj[1] * frame.shape[0])
                width = int(obj[2] * frame.shape[1])
                height = int(obj[3] * frame.shape[0])

                x = int(center_x - width / 2)
                y = int(center_y - height / 2)

                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [ ]:
 cv2.imshow('Object Detection', frame)
     if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()